# Guided Hamiltonian Problem

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `GuidedHamiltonian`
Solve the Guided (Sparse) Hamiltonian Problem.

Definition 4.8 (modified to accept any block-encoding):
In the Guided Hamiltonian problem we are given the following as input:

1. A $(\sqrt{2} s, \cdot, 0)$-block-encoding of a Hamiltonian $H$ such that $\|H\|_\max \le s$.
2. A unitary program that prepares $|\Psi\rangle|0^A\rangle$.
3. Parameters $\lambda \in [-\Lambda, \Lambda]$, $\alpha \in (0, 1)$, $\gamma \in (0, 1]$.

and we should output

- YES (1) if $\| \Pi_{\ge \lambda} (H) |\Psi\rangle \| \ge \gamma$
- NO (0) if $\|H\| \le (1 - \alpha) \lambda$

Note that the above drops the sparsity requirement, and accepts any
$(\alpha_H, \cdot, \cdot)$-block-encoding of $H$.
In the sparse Hamiltonian case, $\alpha_H = s$ (where $s$ is the sparsity).

Algorithm (Theorem 4.9):
    This uses phase estimation on the block-encoding of $e^{iHt}$, and then uses
    amplitude amplification to increase the success probability to $1 - o(1)$.

We instead directly do phase-estimation on the qubitized walk operator for $H$.

#### Parameters
 - `hamiltonian`: the block-encoding of $H$
 - `guiding_state`: the unitary that prepares $|\Psi\rangle$
 - `lambd`: parameter $\lambda$
 - `alpha`: parameter $\alpha$
 - `gamma`: parameter $\gamma$ 

#### References
 - [Quartic quantum speedups for planted inference](https://arxiv.org/abs/2406.19378v1). Section 4.2 "Guided Sparse Hamiltonian Problem".


In [ ]:
from qualtran.bloqs.hamiltonian_simulation import GuidedHamiltonian

### Example Instances

In [ ]:
import sympy

from qualtran.bloqs.optimization.k_xor_sat import GuidingState, KikuchiHamiltonian, KXorInstance
from qualtran.bloqs.state_preparation.black_box_prepare import BlackBoxPrepare
from qualtran.symbolics import ceil, log2

n, k, m, c = sympy.symbols("n k m c", positive=True, integer=True)
zeta = sympy.symbols(r"\zeta", positive=True)

inst_guide = KXorInstance.symbolic(n, (1 - zeta) * m, k, max_rhs=2)
inst_solve = KXorInstance.symbolic(n, zeta * m, k, max_rhs=2)
l = c * k
s = l * ceil(log2(n))

Psi = GuidingState(inst_guide, l)
H = KikuchiHamiltonian(inst_solve, c * k, s)

lambd, alpha, gamma = sympy.symbols(r"\lambda \alpha \gamma", positive=True, real=True)
guided_hamiltonian_symb = GuidedHamiltonian(H, BlackBoxPrepare(Psi), lambd, alpha, gamma)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([guided_hamiltonian_symb],
           ['`guided_hamiltonian_symb`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
guided_hamiltonian_symb_g, guided_hamiltonian_symb_sigma = guided_hamiltonian_symb.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(guided_hamiltonian_symb_g)
show_counts_sigma(guided_hamiltonian_symb_sigma)

## `GuidedHamiltonianPhaseEstimation`
Phase estimation subroutine for the Guided Hamiltonian Problem.

Implements the algorithm $U_\text{PE}$ used in the Guided Hamiltonian Problem.
See the bloq `GuidedHamiltonian` for the full problem definition.

#### References
 - [Quartic quantum speedups for planted inference](https://arxiv.org/abs/2406.19378v1). Section 4.2 "Guided Sparse Hamiltonian Problem".


In [ ]:
from qualtran.bloqs.hamiltonian_simulation import GuidedHamiltonianPhaseEstimation

### Example Instances

In [ ]:
import sympy

from qualtran.bloqs.optimization.k_xor_sat import GuidingState, KikuchiHamiltonian, KXorInstance
from qualtran.bloqs.state_preparation.black_box_prepare import BlackBoxPrepare
from qualtran.symbolics import ceil, log2

n, k, c = sympy.symbols("n k c", positive=True, integer=True)
m_guide, m_solve = sympy.symbols("m_1 m_2", positive=True, integer=True)

inst_guide = KXorInstance.symbolic(n, m_guide, k, max_rhs=2)
inst_solve = KXorInstance.symbolic(n, m_solve, k, max_rhs=2)
l = c * k
s = l * ceil(log2(n))

Psi = GuidingState(inst_guide, l)
H = KikuchiHamiltonian(inst_solve, c * k, s)

eps, delta = sympy.symbols(r"\epsilon_{PE} \delta_{PE}", positive=True, real=True)
guided_phase_estimate_symb = GuidedHamiltonianPhaseEstimation(
    H, BlackBoxPrepare(Psi), eps, delta
)


#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([guided_phase_estimate_symb],
           ['`guided_phase_estimate_symb`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
guided_phase_estimate_symb_g, guided_phase_estimate_symb_sigma = guided_phase_estimate_symb.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(guided_phase_estimate_symb_g)
show_counts_sigma(guided_phase_estimate_symb_sigma)